In [0]:
spark

In [0]:
dbutils.fs.put("/FileStore/tables/channels.csv","""CHANNEL_ID,CHANNEL_DESC,CHANNEL_CLASS,CHANNEL_CLASS_ID,CHANNEL_TOTAL,CHANNEL_TOTAL_ID
3,Direct Sales,Direct,12,Channel total,1
9,Tele Sales,Direct,12,Channel total,1
5,Catalog,Indirect,13,Channel total,1
4,Internet,Indirect,13,Channel total,1
2,Partners,Others,14,Channel total,1
12,Partners,Others,14,Channel total,1,45,ram,3434
sample,Partners,Others,14,Channel total,1,45,ram,3434
10 Partners Others 14 Channel total 1
11 Partners Others 14 Channel total 1""")

# PERMISSIVE (default): nulls are inserted for fields that could not be parsed correctly
# DROPMALFORMED: drops lines that contain fields that could not be parsed
# FAILFAST: aborts the reading if any malformed data is found

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-247691655964474>, line 1
----> 1 dbutils.fs.put("/FileStore/tables/channels.csv","""CHANNEL_ID,CHANNEL_DESC,CHANNEL_CLASS,CHANNEL_CLASS_ID,CHANNEL_TOTAL,CHANNEL_TOTAL_ID
      2 3,Direct Sales,Direct,12,Channel total,1
      3 9,Tele Sales,Direct,12,Channel total,1
      4 5,Catalog,Indirect,13,Channel total,1
      5 4,Internet,Indirect,13,Channel total,1
      6 2,Partners,Others,14,Channel total,1
      7 12,Partners,Others,14,Channel total,1,45,ram,3434
      8 sample,Partners,Others,14,Channel total,1,45,ram,3434
      9 10 Partners Others 14 Channel total 1
     10 11 Partners Others 14 Channel total 1""")

File /databricks/python_shell/dbruntime/dbutils.py:378, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    376 exc.__context__ = None
    377 exc.__cause__ = No

In [0]:
df_channel = spark.read.csv("/FileStore/tables/channels.csv", header=True, inferSchema=True)
df_channel.schema

In [0]:
df_channel.display()

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
sc = StructType([
    StructField('CHANNEL_ID', IntegerType(), True), 
    StructField('CHANNEL_DESC', StringType(), True), 
    StructField('CHANNEL_CLASS', StringType(), True), 
    StructField('CHANNEL_CLASS_ID', IntegerType(), True), 
    StructField('CHANNEL_TOTAL', StringType(), True), 
    StructField('CHANNEL_TOTAL_ID', IntegerType(), True)
])
df_channel1 = spark.read.schema(sc).option("mode", "PERMISSIVE").csv("/FileStore/tables/channels.csv", header=True)
df_channel1.display()

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
sc = StructType([
    StructField('CHANNEL_ID', IntegerType(), True), 
    StructField('CHANNEL_DESC', StringType(), True), 
    StructField('CHANNEL_CLASS', StringType(), True), 
    StructField('CHANNEL_CLASS_ID', IntegerType(), True), 
    StructField('CHANNEL_TOTAL', StringType(), True), 
    StructField('CHANNEL_TOTAL_ID', IntegerType(), True)
])
df_channel1 = spark.read.schema(sc).option("mode", "FAILFAST").csv("/FileStore/tables/channels.csv", header=True)
df_channel1.display()

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
sc = StructType([
    StructField('CHANNEL_ID', IntegerType(), True), 
    StructField('CHANNEL_DESC', StringType(), True), 
    StructField('CHANNEL_CLASS', StringType(), True), 
    StructField('CHANNEL_CLASS_ID', IntegerType(), True), 
    StructField('CHANNEL_TOTAL', StringType(), True), 
    StructField('CHANNEL_TOTAL_ID', IntegerType(), True)
])
df_channel1 = spark.read.schema(sc).option("mode", "DROPMALFORMED").csv("/FileStore/tables/channels.csv", header=True)
df_channel1.display()

In [0]:
# another way to handle bad recods is store them at particular location
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
sc = StructType([
    StructField('CHANNEL_ID', IntegerType(), True), 
    StructField('CHANNEL_DESC', StringType(), True), 
    StructField('CHANNEL_CLASS', StringType(), True), 
    StructField('CHANNEL_CLASS_ID', IntegerType(), True), 
    StructField('CHANNEL_TOTAL', StringType(), True), 
    StructField('CHANNEL_TOTAL_ID', IntegerType(), True)
])
df_channel1 = spark.read.schema(sc).option("badRecordsPath", "/channels/badata/").csv("/FileStore/tables/channels.csv", header=True)
df_channel1.display()

CHANNEL_ID,CHANNEL_DESC,CHANNEL_CLASS,CHANNEL_CLASS_ID,CHANNEL_TOTAL,CHANNEL_TOTAL_ID
3,Direct Sales,Direct,12,Channel total,1
9,Tele Sales,Direct,12,Channel total,1
5,Catalog,Indirect,13,Channel total,1
4,Internet,Indirect,13,Channel total,1
2,Partners,Others,14,Channel total,1


In [0]:
%fs
ls /channels/badata/20231211T092420/bad_records/

path,name,size,modificationTime
dbfs:/channels/badata/20231211T092420/bad_records/part-00000-bd4d11a1-c38b-4204-a163-bfd1355d278b,part-00000-bd4d11a1-c38b-4204-a163-bfd1355d278b,1040,1702286662000


In [0]:
%fs 
head dbfs:/channels/badata/20231211T092420/bad_records/part-00000-bd4d11a1-c38b-4204-a163-bfd1355d278b

{"path":"dbfs:/FileStore/tables/channels.csv","record":"12,Partners,Others,14,Channel total,1,45,ram,3434","reason":"org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 12,Partners,Others,14,Channel total,1,45,ram,3434 SQLSTATE: KD000"}
{"path":"dbfs:/FileStore/tables/channels.csv","record":"sample,Partners,Others,14,Channel total,1,45,ram,3434","reason":"org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: sample,Partners,Others,14,Channel total,1,45,ram,3434 SQLSTATE: KD000"}
{"path":"dbfs:/FileStore/tables/channels.csv","record":"10 Partners Others 14 Channel total 1","reason":"org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 10 Partners Others 14 Channel total 1 SQLSTATE: KD000"}
{"path":"dbfs:/FileStore/tables/channels.csv","record":"11 Partners Others 14 Channel total 1","reason":"org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 11 Partners Others 14 Channel total 1 SQLSTATE: KD000"}

In [0]:
# here we check how our bad data and it's meta data get store in json format
df_badata = spark.read.json("/channels/badata/*/*/")
df_badata.display()

path,reason,record
dbfs:/FileStore/tables/channels.csv,"org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 12,Partners,Others,14,Channel total,1,45,ram,3434 SQLSTATE: KD000","12,Partners,Others,14,Channel total,1,45,ram,3434"
dbfs:/FileStore/tables/channels.csv,"org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: sample,Partners,Others,14,Channel total,1,45,ram,3434 SQLSTATE: KD000","sample,Partners,Others,14,Channel total,1,45,ram,3434"
dbfs:/FileStore/tables/channels.csv,org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 10 Partners Others 14 Channel total 1 SQLSTATE: KD000,10 Partners Others 14 Channel total 1
dbfs:/FileStore/tables/channels.csv,org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 11 Partners Others 14 Channel total 1 SQLSTATE: KD000,11 Partners Others 14 Channel total 1


In [0]:
# another method is create a column and store badate in that column
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
sc = StructType([
    StructField('CHANNEL_ID', IntegerType(), True), 
    StructField('CHANNEL_DESC', StringType(), True), 
    StructField('CHANNEL_CLASS', StringType(), True), 
    StructField('CHANNEL_CLASS_ID', IntegerType(), True), 
    StructField('CHANNEL_TOTAL', StringType(), True), 
    StructField('CHANNEL_TOTAL_ID', IntegerType(), True),
    StructField('Bad_DATA', StringType(), True)
])
df_channel1 = spark.read.schema(sc).csv("/FileStore/tables/channels.csv", header=True, columnNameOfCorruptRecord="Bad_DATA")
df_channel1.display()

CHANNEL_ID,CHANNEL_DESC,CHANNEL_CLASS,CHANNEL_CLASS_ID,CHANNEL_TOTAL,CHANNEL_TOTAL_ID,Bad_DATA
3,Direct Sales,Direct,12,Channel total,1,null
9,Tele Sales,Direct,12,Channel total,1,null
5,Catalog,Indirect,13,Channel total,1,null
4,Internet,Indirect,13,Channel total,1,null
2,Partners,Others,14,Channel total,1,null
12,Partners,Others,14,Channel total,1,"12,Partners,Others,14,Channel total,1,45,ram,3434"
null,Partners,Others,14,Channel total,1,"sample,Partners,Others,14,Channel total,1,45,ram,3434"
null,null,null,null,null,null,10 Partners Others 14 Channel total 1
null,null,null,null,null,null,11 Partners Others 14 Channel total 1


In [0]:
# filtering good data and bad data
good_data = df_channel1.filter("Bad_DATA is null").drop("Bad_DATA")
print("Good Data -> ")
good_data.display()

Good Data -> 


CHANNEL_ID,CHANNEL_DESC,CHANNEL_CLASS,CHANNEL_CLASS_ID,CHANNEL_TOTAL,CHANNEL_TOTAL_ID
3,Direct Sales,Direct,12,Channel total,1
9,Tele Sales,Direct,12,Channel total,1
5,Catalog,Indirect,13,Channel total,1
4,Internet,Indirect,13,Channel total,1
2,Partners,Others,14,Channel total,1


In [0]:
df_channel1.filter("Bad_DATA is not null").display()

CHANNEL_ID,CHANNEL_DESC,CHANNEL_CLASS,CHANNEL_CLASS_ID,CHANNEL_TOTAL,CHANNEL_TOTAL_ID,Bad_DATA
12,Partners,Others,14,Channel total,1,"12,Partners,Others,14,Channel total,1,45,ram,3434"
null,Partners,Others,14,Channel total,1,"sample,Partners,Others,14,Channel total,1,45,ram,3434"
null,null,null,null,null,null,10 Partners Others 14 Channel total 1
null,null,null,null,null,null,11 Partners Others 14 Channel total 1


In [0]:
bad_data = df_channel1.filter("Bad_DATA is not null").select("Bad_DATA")
print("Bad Data -> ")
bad_data.display()

# i don't know why output is not shown here

Bad Data -> 


Bad_DATA
